In [ ]:
# Code source: Sebastian Curi and Andreas Krause.

# Python Notebook Commands
%matplotlib inline
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

# Numerical Libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = (10, 5)   # Change this if figures look ugly. 
import time 
import copy 

# IPython Libraries
import IPython
import ipywidgets
from ipywidgets import interact, interactive, interact_manual
from IPython.display import clear_output


# SKLEARN Libraries
from sklearn import cluster, datasets, mixture



In [ ]:
class LloydsAlgorithm(object):
    """Apply Loyd's algorithm initializing the means uniformly at random from a 
    Uniform distribution over the min/max points independently for each coordinate 
    in the domain."""
    def __init__(self, num_centers, X):
        self.X = X
        self.num_points, self.dimension = X.shape

        self.num_centers = num_centers
        self.restart()
        
    def restart(self):
        min_, max_ = np.min(self.X, axis=0), np.max(self.X, axis=0)        
        self.means =  np.random.uniform(low=min_, high=max_, size=(self.num_centers, self.dimension))
        self.centers = None
    
    def update_means(self):
        for k in range(self.num_centers):
            idx = np.where(self.centers == k)[0]
            if len(idx) > 0:
                self.means[k] = self.X[idx].mean(axis=0)
    
    def assign_centers(self):
        if self.centers is None:
            self.centers = np.zeros(self.num_points, dtype=np.int)
            
        for i, x in enumerate(self.X):
            d = np.linalg.norm(x - self.means, 2, axis=1) ** 2  
            self.centers[i] = np.argmin(d)
            
    def run(self, max_iter):
        converged = False 
        old_means = copy.deepcopy(self.means)

        i = 0 
        while not converged:
            i += 1
            self.assign_centers()     
            self.update_means()

            if i > max_iter:
                break 
            converged = np.all(self.means == old_means)
            old_means = copy.deepcopy(self.means)
            
            
class KMeans(LloydsAlgorithm):
    """Apply Lloyd's Algorithm initializing the means uniformly at random from the data set."""
    def restart(self):
        idx = np.random.choice(self.num_points, self.num_centers, replace=False)
        self.means = self.X[idx]
        self.centers = None
        
class KMeansPP(LloydsAlgorithm):
    """Apply Lloyd's Algorithm initializing the means uniformly at random from the data set."""
    def restart(self):
        self.means = np.empty((self.num_centers, self.dimension))
        i = np.random.choice(self.num_points, 1)
        self.means[0] = self.X[i]
        indexes = [i] 
        for j in range(1, self.num_centers):
            probs = np.zeros(self.num_points)
            
            for i, x in enumerate(self.X):
                d = np.linalg.norm(x - self.means[:j], 2, axis=1) ** 2
                probs[i] = np.min(d)
            
            probs = probs / np.sum(probs)
            i = np.random.choice(self.num_points, 1, p = probs)
            assert np.all(probs[np.array(indexes)] == 0)
            
            self.means[j] = self.X[i] 
            indexes.append(i)
        self.centers = None
        
        
def k_means_cost(means, X):
    cost = 0
    for x in X:
        d = np.linalg.norm(x - means, 2, axis=1) ** 2
        cost += np.min(d)
    return cost


In [ ]:
def run_lloyds(dataset, num_centers, init):
    n_samples = 200
    # Generate Data
    if dataset == '3-blobs':
        X, y = datasets.make_blobs(n_samples=n_samples, random_state=8)
    elif dataset == '4-blobs':
        X, y = datasets.make_blobs(centers=4, n_samples=n_samples, random_state=8)
    elif dataset == 'circles':
        X, Y = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05)
    elif dataset == 'moons':
        X, Y = datasets.make_moons(n_samples=n_samples, noise=.05)
    elif dataset == 'no_structure':
        X, Y = np.random.rand(n_samples, 2), None 
    elif dataset == 'anisotropic':
        X, Y = datasets.make_blobs(n_samples=n_samples, random_state=170)
        transformation = [[0.6, -0.6], [-0.4, 0.8]]
        X = np.dot(X, transformation)
    elif dataset == 'varied variance':
        X, Y = datasets.make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=170)
    elif dataset == 'no structure':
        X, Y = np.random.rand(n_samples, 2), None 

    
    # Initialize algorithm
    if init == 'random':
        alg = LloydsAlgorithm(num_centers=num_centers, X=X)
    elif init == 'random from data set':
        alg = KMeans(num_centers=num_centers, X=X)
    elif 'kmeans++':
        alg = KMeansPP(num_centers=num_centers, X=X)
        
    button = ipywidgets.Button(description="Assign Center")
    button2 = ipywidgets.Button(description="Update Means")
    button3 = ipywidgets.Button(description="Restart Algorithm")
    button4 = ipywidgets.Button(description="Run Algorithm")

    
    def plot_lloyds():
        colors = np.array(['#377eb8', '#ff7f00', 
                           '#f781bf', '#a65628', '#984ea3',
                           '#999999', '#e41a1c', '#dede00', 
                           '#000000', '#4daf4a'][0:num_centers+1])
        
        if alg.centers is not None:
            plt.scatter(alg.X[:, 0], alg.X[:, 1], marker='.', color=colors[alg.centers])
        else:
            plt.scatter(alg.X[:, 0], alg.X[:, 1], marker='.', color='g')

        plt.scatter(alg.means[:, 0], alg.means[:, 1], marker='*', color=colors[:num_centers], s=250)
        
        plt.title(f"Cost: {k_means_cost(alg.means, alg.X)}")
        IPython.display.clear_output(wait=True)
        IPython.display.display(plt.gcf())
        plt.close()

        display(button);
        display(button2);
        display(button3);
        display(button4);

    
    def restart(b):
        alg.restart()
        plot_lloyds()
        
    def assign_centers(b):
        alg.assign_centers()
        plot_lloyds()

    def update_means(b):
        alg.update_means()
        plot_lloyds()
    
    def run(b):
        converged = False 
        old_means = copy.deepcopy(alg.means)
        max_iter = 10
        i = 0 
        while not converged:
            i += 1
            alg.assign_centers()
            plot_lloyds()
            time.sleep(0.5)
            
            alg.update_means()
            plot_lloyds()
            time.sleep(0.5)

            if i > max_iter:
                break 
            converged = np.all(alg.means == old_means)
            old_means = copy.deepcopy(alg.means)
            
    restart(None)
    button.on_click(assign_centers)
    button2.on_click(update_means)
    button3.on_click(restart)
    button4.on_click(run)

    plot_lloyds()

n_centers_widget = ipywidgets.IntSlider(value=2, min=1, max=10, step=1,  description='Number of Centers:', 
                                         style={'description_width': 'initial'}, continuous_update=False)
dataset_widget = ipywidgets.Dropdown(
    value='3-blobs', 
    options=['3-blobs', '4-blobs', 'circles', 'moons', 'anisotropic', 'varied variance', 'no structure'], 
    description='Dataset:',  style={'description_width': 'initial'}, continuous_update=False)
initialization_widget = ipywidgets.Dropdown(
    value='random', 
    options=['random', 'random from data set', 'kmeans++'], 
    description='Initialization:',  style={'description_width': 'initial'}, continuous_update=False)


interact(run_lloyds, dataset=dataset_widget, init=initialization_widget, num_centers=n_centers_widget);

# How to Select k?

In [ ]:
def determine_k(dataset, init, lambda_):
    n_samples = 200
    # Generate Data
    if dataset == '3-blobs':
        X, y = datasets.make_blobs(n_samples=n_samples, random_state=8)
    elif dataset == '4-blobs':
        X, y = datasets.make_blobs(centers=4, n_samples=n_samples, random_state=8)
    elif dataset == 'circles':
        X, Y = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05)
    elif dataset == 'moons':
        X, Y = datasets.make_moons(n_samples=n_samples, noise=.05)
    elif dataset == 'no_structure':
        X, Y = np.random.rand(n_samples, 2), None 
    elif dataset == 'anisotropic':
        X, Y = datasets.make_blobs(n_samples=n_samples, random_state=170)
        transformation = [[0.6, -0.6], [-0.4, 0.8]]
        X = np.dot(X, transformation)
    elif dataset == 'varied variance':
        X, Y = datasets.make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=170)
    elif dataset == 'no structure':
        X, Y = np.random.rand(n_samples, 2), None 

        
    K = 10
    costs = np.zeros(K)
    for k in range(K):
        # Initialize algorithm
        if init == 'random':
            alg = LloydsAlgorithm(num_centers=k + 1, X=X)
        elif init == 'random from data set':
            alg = KMeans(num_centers=k + 1, X=X)
        elif 'kmeans++':
            alg = KMeansPP(num_centers=k + 1, X=X)     
            
        alg.run(100)
        costs[k] = k_means_cost(alg.means, alg.X) / X.shape[0]
    
    reg_cost = costs + lambda_ * (np.arange(K) + 1) 
#     fig, ax = plt.subplots(2, 1)
    plt.plot(np.arange(K) + 1, reg_cost)
    plt.xlabel('Number of centers.')
    plt.ylabel('K-Means Cost');
    plt.show()
    
    # GET BEST K
    k = np.argmin(reg_cost)
    if init == 'random':
        alg = LloydsAlgorithm(num_centers=k + 1, X=X)
    elif init == 'random from data set':
        alg = KMeans(num_centers=k + 1, X=X)
    elif 'kmeans++':
        alg = KMeansPP(num_centers=k + 1, X=X)     

    alg.run(100)
    
    colors = np.array(['#377eb8', '#ff7f00', 
                   '#f781bf', '#a65628', '#984ea3',
                   '#999999', '#e41a1c', '#dede00', 
                   '#000000', '#4daf4a'][0:k+1])
    
    plt.scatter(alg.X[:, 0], alg.X[:, 1], marker='.', color=colors[alg.centers])
    plt.scatter(alg.means[:, 0], alg.means[:, 1], marker='*', color=colors[:(k + 1)], s=250)
    plt.title('Optimal k-Means')
    plt.show()
    

dataset_widget = ipywidgets.Dropdown(
    value='3-blobs', 
    options=['3-blobs', '4-blobs', 'circles', 'moons', 'anisotropic', 'varied variance', 'no structure'], 
    description='Dataset:',  style={'description_width': 'initial'}, continuous_update=False)
initialization_widget = ipywidgets.Dropdown(
    value='random', 
    options=['random', 'random from data set', 'kmeans++'], 
    description='Initialization:',  style={'description_width': 'initial'}, continuous_update=False)
lambda_widget = ipywidgets.FloatSlider(
    value=0, min=0, max=10, step=0.1, 
    description='Regularization Parameter:',  style={'description_width': 'initial'}, continuous_update=False)
interact(determine_k, dataset=dataset_widget, init=initialization_widget, lambda_=lambda_widget);